In [4]:
#Author: Varun Ram S
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline 

In [5]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
from keras.callbacks import EarlyStopping

In [7]:
pm_train=pd.read_csv('/content/PM_train.txt', sep=' ',header=None).drop([26,27],axis=1)
feature_names = ['id','cycle','setting1','setting2','setting3','para1','para2','para3','para4','para5','para6','para7','para8','para9','para10','para11','para12','para13','para14','para15','para16','para17','para18','para19','para20','para21']
pm_train.columns=feature_names
print('Shape of Train dataset: ',pm_train.shape)
pm_train.head()

Shape of Train dataset:  (20631, 26)


,id,cycle,setting1,setting2,setting3,para1,para2,para3,para4,para5,...,para12,para13,para14,para15,para16,para17,para18,para19,para20,para21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


In [8]:
pm_test=pd.read_csv('/content/PM_test.txt',sep=' ',header=None).drop([26,27],axis=1)
pm_test.columns=feature_names
#dataset_test.head()
print('Shape of Test dataset: ',pm_test.shape)
pm_test.head()

Shape of Test dataset:  (12323, 26)


,id,cycle,setting1,setting2,setting3,para1,para2,para3,para4,para5,...,para12,para13,para14,para15,para16,para17,para18,para19,para20,para21
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,521.72,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,522.16,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,521.97,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,521.38,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,522.15,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130


In [9]:
pm_truth=pd.read_csv('/content/PM_truth.txt',sep=' ',header=None).drop([1],axis=1)
pm_truth.columns=['mg']
pm_truth['id']=pm_truth.index+1
pm_truth.head()

,mg,id
0,112,1
1,98,2
2,69,3
3,82,4
4,91,5


In [10]:
r=pd.DataFrame(pm_test.groupby('id')['cycle'].max()).reset_index()
r.columns = ['id', 'max']
r.head()

,id,max
0,1,31
1,2,49
2,3,126
3,4,106
4,5,98


In [11]:
pm_truth['rtf']=pm_truth['mg']+r['max']
pm_truth.head()

,mg,id,rtf
0,112,1,143.0
1,98,2,147.0
2,69,3,195.0
3,82,4,188.0
4,91,5,189.0


In [12]:
pm_truth.drop('mg', axis=1, inplace=True)
pm_test=pm_test.merge(pm_truth,on=['id'],how='left')
pm_test['ttf']=pm_test['rtf'] - pm_test['cycle']
pm_test.drop('rtf', axis=1, inplace=True)
pm_test.head()

,id,cycle,setting1,setting2,setting3,para1,para2,para3,para4,para5,...,para13,para14,para15,para16,para17,para18,para19,para20,para21,ttf
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735,142.0
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916,141.0
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166,140.0
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737,139.0
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130,138.0


In [13]:
pm_train['ttf'] = pm_train.groupby(['id'])['cycle'].transform(max)-pm_train['cycle']
pm_train.head()

,id,cycle,setting1,setting2,setting3,para1,para2,para3,para4,para5,...,para13,para14,para15,para16,para17,para18,para19,para20,para21,ttf
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187


In [14]:
df_train=pm_train.copy()
df_test=pm_train.copy()
period=30
df_train['label_bc'] = df_train['ttf'].apply(lambda x: 1 if x <= period else 0)
df_test['label_bc'] = df_test['ttf'].apply(lambda x: 1 if x <= period else 0)
df_train.head()

,id,cycle,setting1,setting2,setting3,para1,para2,para3,para4,para5,...,para14,para15,para16,para17,para18,para19,para20,para21,ttf,label_bc
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191,0
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190,0
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189,0
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188,0
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187,0


In [15]:
features_col_name=['setting1','setting2','setting3','para1','para2','para3','para4','para5','para6','para7','para8','para9','para10','para11','para12','para13','para14','para15','para16','para17','para18','para19','para20','para21']
target_col_name='label_bc'

In [17]:
sc=MinMaxScaler()
df_train[features_col_name]=sc.fit_transform(df_train[features_col_name])
df_test[features_col_name]=sc.transform(df_test[features_col_name])

In [18]:
def gen_sequence(id_df, seq_length, seq_cols):
    df_zeros=pd.DataFrame(np.zeros((seq_length-1,id_df.shape[1])),columns=id_df.columns)
    id_df=df_zeros.append(id_df,ignore_index=True)
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    lstm_array=[]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        lstm_array.append(data_array[start:stop, :])
    return np.array(lstm_array)

# function to generate labels
def gen_label(id_df, seq_length, seq_cols,label):
    df_zeros=pd.DataFrame(np.zeros((seq_length-1,id_df.shape[1])),columns=id_df.columns)
    id_df=df_zeros.append(id_df,ignore_index=True)
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    y_label=[]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        y_label.append(id_df[label][stop])
    return np.array(y_label)

In [19]:
seq_length=50
seq_cols=features_col_name

In [20]:
# generate X_train
X_train=np.concatenate(list(list(gen_sequence(df_train[df_train['id']==id], seq_length, seq_cols)) for id in df_train['id'].unique()))
print(X_train.shape)
# generate y_train
y_train=np.concatenate(list(list(gen_label(df_train[df_train['id']==id], 50, seq_cols,'label_bc')) for id in df_train['id'].unique()))
print(y_train.shape)

<ipython-input-18-ce390dc8d101>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_df=df_zeros.append(id_df,ignore_index=True)
<ipython-input-18-ce390dc8d101>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_df=df_zeros.append(id_df,ignore_index=True)
<ipython-input-18-ce390dc8d101>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_df=df_zeros.append(id_df,ignore_index=True)
<ipython-input-18-ce390dc8d101>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_df=df_zeros.append(id_df,ignore_index=True)
<ipython-input-18-ce390dc8d101>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future 

(20531, 50, 24)


<ipython-input-18-ce390dc8d101>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_df=df_zeros.append(id_df,ignore_index=True)
<ipython-input-18-ce390dc8d101>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_df=df_zeros.append(id_df,ignore_index=True)
<ipython-input-18-ce390dc8d101>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_df=df_zeros.append(id_df,ignore_index=True)
<ipython-input-18-ce390dc8d101>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_df=df_zeros.append(id_df,ignore_index=True)
<ipython-input-18-ce390dc8d101>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fu

(20531,)


<ipython-input-18-ce390dc8d101>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_df=df_zeros.append(id_df,ignore_index=True)
<ipython-input-18-ce390dc8d101>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_df=df_zeros.append(id_df,ignore_index=True)
<ipython-input-18-ce390dc8d101>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_df=df_zeros.append(id_df,ignore_index=True)
<ipython-input-18-ce390dc8d101>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_df=df_zeros.append(id_df,ignore_index=True)
<ipython-input-18-ce390dc8d101>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fu

In [21]:
X_test=np.concatenate(list(list(gen_sequence(df_test[df_test['id']==id], seq_length, seq_cols)) for id in df_test['id'].unique()))
print(X_test.shape)

y_test=np.concatenate(list(list(gen_label(df_test[df_test['id']==id], 50, seq_cols,'label_bc')) for id in df_test['id'].unique()))
print(y_test.shape)

<ipython-input-18-ce390dc8d101>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_df=df_zeros.append(id_df,ignore_index=True)
<ipython-input-18-ce390dc8d101>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_df=df_zeros.append(id_df,ignore_index=True)
<ipython-input-18-ce390dc8d101>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_df=df_zeros.append(id_df,ignore_index=True)
<ipython-input-18-ce390dc8d101>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_df=df_zeros.append(id_df,ignore_index=True)
<ipython-input-18-ce390dc8d101>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future 

(20531, 50, 24)


<ipython-input-18-ce390dc8d101>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_df=df_zeros.append(id_df,ignore_index=True)
<ipython-input-18-ce390dc8d101>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_df=df_zeros.append(id_df,ignore_index=True)
<ipython-input-18-ce390dc8d101>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_df=df_zeros.append(id_df,ignore_index=True)
<ipython-input-18-ce390dc8d101>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_df=df_zeros.append(id_df,ignore_index=True)
<ipython-input-18-ce390dc8d101>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fu

(20531,)


<ipython-input-18-ce390dc8d101>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_df=df_zeros.append(id_df,ignore_index=True)
<ipython-input-18-ce390dc8d101>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_df=df_zeros.append(id_df,ignore_index=True)
<ipython-input-18-ce390dc8d101>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_df=df_zeros.append(id_df,ignore_index=True)
<ipython-input-18-ce390dc8d101>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_df=df_zeros.append(id_df,ignore_index=True)
<ipython-input-18-ce390dc8d101>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fu

In [22]:
nb_features =X_train.shape[2]
timestamp=seq_length

model = Sequential()

model.add(LSTM(
         input_shape=(timestamp, nb_features),
         units=100,
         return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(
          units=50,
          return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 100)           50000     
                                                                 
 dropout (Dropout)           (None, 50, 100)           0         
                                                                 
 lstm_1 (LSTM)               (None, 50)                30200     
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 80,251
Trainable params: 80,251
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.fit(X_train, y_train, epochs=20, batch_size=200, validation_split=0.05, verbose=1,
          callbacks = [EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')])

Epoch 1/20
98/98 [==============================] - 31s 312ms/step - loss: 0.0486 - accuracy: 0.9782 - val_loss: 0.0534 - val_accuracy: 0.9727
Epoch 2/20
98/98 [==============================] - 29s 297ms/step - loss: 0.0513 - accuracy: 0.9777 - val_loss: 0.0380 - val_accuracy: 0.9805
Epoch 3/20
98/98 [==============================] - 30s 306ms/step - loss: 0.0520 - accuracy: 0.9783 - val_loss: 0.0551 - val_accuracy: 0.9737


In [34]:
scores = model.evaluate(X_train, y_train, verbose=1, batch_size=200)
print('Accurracy: {}'.format(scores[1]))

103/103 [==============================] - 11s 108ms/step - loss: 0.0443 - accuracy: 0.9813
Accurracy: 0.9812965989112854


In [35]:
def prob_Machine_failure(machine_id):
    machine_df=df_test[df_test.id==machine_id]
    machine_test=gen_sequence(machine_df,seq_length,seq_cols)
    m_pred=model.predict(machine_test)
    failure_prob=list(m_pred[-1]*100)[0]
    return failure_prob

In [36]:
machine_id=13
print('Probability of machine failing within 30 days: ',prob_Machine_failure(machine_id))

4/6 [===================>..........] - ETA: 0s

<ipython-input-18-ce390dc8d101>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_df=df_zeros.append(id_df,ignore_index=True)


6/6 [==============================] - 0s 22ms/step
Probability of machine failing within 30 days:  99.97322
